## Import Libraries

In [1]:
import sys
import tensorflow.keras
import tensorflow as tf

In [2]:
# keras module for building LSTM 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

## Data Preprocessing

 Read Data

In [22]:
path=r"C:\Users\Shibbs\Desktop\Praxis\CAPP\cwd\lstm_datasets\lstm_with_seq_100.csv"

In [23]:
df = pd.read_csv(path)

In [24]:
df.seq_index = df.seq_index.astype(int)  
df = df[df['seq_index']<30].reset_index()

In [25]:
df3 = pd.pivot_table(data=df,index='user_id',columns=['seq_index'],values=['event_type'],aggfunc='sum')
df3 = df3.fillna(0)
df3 = df3[df3['event_type', 6]!= 0]

In [26]:
df3

event_type                                                        \
seq_index            0     1     2     3     4     5     6     7     8     9    
user_id                                                                         
1.515916e+18       view  view  view  view  view  view  view  view  view  view   
1.515916e+18       view  view  view  view  view  view  view  view  view  view   
1.515916e+18       view  view  view  view  view  view  view  view  view  view   
1.515916e+18       view  view  view  view  view  view  view  view  view  view   
1.515916e+18       view  view  view  view  view  view  view  view  view  view   
...                 ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
1.515916e+18       view  view  view  view  view  view  view  view  view  view   
1.515916e+18       view  view  view  view  view  view  view  view  view  cart   
1.515916e+18       view  view  view  view  view  view  view  view  view  view   
1.515916e+18       view  view  view  view  view  view  view  view  view  view   
1.515920e+18       view  view  view  view  view  view  view  view  view  view   

              ...                                                              \
seq_index     ...    20        21    22    23        24        25    26    27   
user_id       ...                                                               
1.515916e+18  ...  view      view  view  view      view      view  view  view   
1.515916e+18  ...  cart  purchase  view  view      view      view  view  view   
1.515916e+18  ...  view      view  view  view      view      view  view  view   
1.515916e+18  ...  view      view  view  view         0         0     0     0   
1.515916e+18  ...  view      view  view  view      view      view  view  view   
...           ...   ...       ...   ...   ...       ...       ...   ...   ...   
1.515916e+18  ...  view      view  view  view  purchase  purchase  view  view   
1.515916e+18  ...  view      view  view  view      view      view  view  view   
1.515916e+18  ...  view      view  view  view      view      view  cart  view   
1.515916e+18  ...  view      view  view  view      view      view  view  view   
1.515920e+18  ...  view      view  view  view      view      view  view  view   

                              
seq_index           28    29  
user_id                       
1.515916e+18      view  view  
1.515916e+18      view  view  
1.515916e+18      view  view  
1.515916e+18         0     0  
1.515916e+18      view  view  
...                ...   ...  
1.515916e+18      view  cart  
1.515916e+18      view  view  
1.515916e+18  purchase  view  
1.515916e+18      view  view  
1.515920e+18      view  view  

[7806 rows x 30 columns]

In [27]:
product_id_list = []
for i in range(len(df3.index)):
    product_id = df3.iloc[i].tolist()
#     product_id.sort(reverse = True,key=lambda x: x==0)
    product_id_list.append(product_id)
    product_id_list = [[ele for ele in sub if ele != 0] for sub in product_id_list]

# product_id_list = np.array(product_id_list)
# pid = product_id_list.flatten()
# pid = pid.astype(int)
# pid = pid.astype(str)
# pid = pid.reshape(int(191520/20),20)
# pid = pid.tolist()

In [28]:
product_id_list=[list( map(str,i) ) for i in product_id_list] 

In [29]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(product_id_list)
inp_sequences[:100]

[[1, 1],
 [1, 1, 1],
 [1, 1, 1, 1],
 [1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [30]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = np_utils.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [31]:
predictors.shape

(198782, 29)

In [32]:
label.shape

(198782, 4)

In [33]:
total_words

4

In [34]:
max_sequence_len

30

In [43]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 4, input_length=input_len))
    model.add(SpatialDropout1D(0.2))
    model.add(Conv1D(64,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(32,
                     5,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(20, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(LSTM(20))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 29, 4)             16        
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 29, 4)            0         
 lDropout1D)                                                     
                                                                 
 conv1d_2 (Conv1D)           (None, 27, 64)            832       
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 13, 64)           0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 9, 32)             10272     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 4, 32)           

In [44]:
model.fit(predictors, label,validation_split=0.3 ,epochs=3, verbose=1, shuffle=False)

Epoch 1/3
4349/4349 [==============================] - 52s 11ms/step - loss: 0.3191 - accuracy: 0.9258 - val_loss: 0.3294 - val_accuracy: 0.9201
Epoch 2/3
4349/4349 [==============================] - 49s 11ms/step - loss: 0.2964 - accuracy: 0.9300 - val_loss: 0.3317 - val_accuracy: 0.9201
Epoch 3/3
4349/4349 [==============================] - 49s 11ms/step - loss: 0.2959 - accuracy: 0.9300 - val_loss: 0.3270 - val_accuracy: 0.9201


In [98]:
model.save('model_93.h5')

In [3]:
from keras.models import load_model
model = load_model('model_93.h5')

In [41]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    tokenizer = Tokenizer()
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences(seed_text)[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        print((predicted))
#         output_word = ""
#         for word,index in tokenizer.word_index.items():
#             if index == np.argmax(predicted):
#                 output_word = word
#                 break
#             seed_text += " "+output_word
#     return seed_text

In [42]:
(generate_text(inp, 
                     1, 
                     model, max_sequence_len=30))


[[8.1114054e-10 9.6504509e-01 3.1636573e-02 3.3183820e-03]]


In [40]:
inp=[1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
inp = [str(x) for x in inp]